# Embedding

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version="2024-10-21",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

In [42]:
response = client.embeddings.create(
    model="text-embedding-3-large",
    input="Hola mundo"
)

vector = response.data[0].embedding
print("Vector length:", len(vector))

Vector length: 3072


## Tokenizer y función para chunkear

In [43]:
import tiktoken

# Usar el nombre de deployment/modelo (definido en la celda de configuración)
# AZURE_DEPLOYMENT debe existir en el entorno de ejecución (se definió en la celda anterior)
MODEL_FOR_ENCODING = globals().get("AZURE_DEPLOYMENT", "text-embedding-3-large")

# Intentar obtener el encoding para el modelo desplegado; si falla, caer a cl100k_base
try:
    ENCODER = tiktoken.encoding_for_model(MODEL_FOR_ENCODING)
except Exception:
    ENCODER = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    return len(ENCODER.encode(text))

def chunk_text(text, max_tokens=450, overlap=50):
    """Divide el texto en chunks usando el encoder del modelo.
    Usa ventana deslizante con `overlap` tokens solapados entre chunks.
    """
    tokens = ENCODER.encode(text)
    chunks = []

    if max_tokens <= 0:
        return [text]

    start = 0
    while start < len(tokens):
        chunk_tokens = tokens[start:start+max_tokens]
        chunk_text = ENCODER.decode(chunk_tokens)
        chunks.append(chunk_text)
        # avanzar con solapamiento
        start += max_tokens - overlap

    return chunks


## Función para generar embeddings

In [44]:
def embed_text(text: str):
    """
    Generates embeddings using Azure OpenAI (2025 syntax).
    Accepts a string or list of strings.
    Returns a vector (list of floats).
    """
    response = client.embeddings.create(
        model="text-embedding-3-large", 
        input=text
    )

    # For a single input, return the 1 vector
    return response.data[0].embedding



## Función general para procesar cada intervención

In [45]:
def process_speech_turn(turn, max_tokens=450):
    """
    Procesa una intervención del discurso:
    - Si es pequeña (<= max_tokens): genera embedding directamente.
    - Si es grande (> max_tokens): la divide en chunks y genera embeddings para cada chunk
    Args:
        turn (dict): Diccionario con la intervención del discurso.
        max_tokens (int): Número máximo de tokens por chunk.
    Returns:
        list: Lista de diccionarios con embeddings y metadatos.
    """
    text = turn["text"]
    token_count = count_tokens(text)

    # Si la intervención es pequeña, solo 1 chunk
    if token_count <= max_tokens:
        embedding = embed_text(text)
        return [{
            "doc_id": turn["doc_id"],
            "sequence": turn["sequence"],
            "chunk_id": None,                      # no hay subdivisión
            "type": turn["type"],
            "speaker": turn["speaker"],
            "text": text,
            "embedding": embedding,
            "token_count": token_count
        }]

    # Si es grande → dividir en chunks
    chunks = chunk_text(text, max_tokens)
    results = []

    for idx, chunk in enumerate(chunks, start=1):
        embedding = embed_text(chunk)

        results.append({
            "doc_id": turn["doc_id"],
            "sequence": turn["sequence"],
            "chunk_id": idx,                     # 1, 2, 3…
            "type": turn["type"],
            "speaker": turn["speaker"],
            "text": chunk,
            "embedding": embedding,
            "token_count": count_tokens(chunk)
        })

    return results


## Apply to an intervention

In [46]:
# Import json file 
import json
with open('2025-01-15-pacic-event-structured.json', 'r', encoding='utf-8') as f:
    conference_sample = json.load(f)

# print json
turn_test = conference_sample[24]
print(turn_test)

{'doc_id': '2025-01-15-pacic-event', 'sequence': 25, 'type': 'speech_turn', 'speaker': {'raw': 'PRESIDENTA DE MÉXICO, CLAUDIA SHEINBAUM PARDO', 'normalized': 'Claudia Sheinbaum Pardo', 'role': 'Presidenta De México'}, 'text': '—El balón—. DIRECTOR GENERAL DE LA COMISIÓN NACIONAL DE CULTURA FÍSICA Y DEPORTE (CONADE), ROMMEL PACHECO MARRUFO: Le paso el balón al maestro Zoé Robledo. DIRECTOR GENERAL INSTITUTO MEXICANO DEL SEGURO SOCIAL (IMSS), ZOÉ ROBLEDO ABURTO: Muchas gracias. Muy buenos días, Presidenta. Muy buenos días a todos y a todas. En el IMSS estamos preparando 3 Mundialitos y una Copa Internacional con un enfoque: nuevas disciplinas y grupos de población que normalmente no encuentran espacio en el futbol tradicional. La idea es recuperar y ampliar la oferta deportiva en instalaciones del IMSS, y aprovechar el impulso del Mundial para que su práctica crezca y permanezca en nuestro país. El primero es un Torneo de Futsal Femenil, organizado por el IMSS y también por la Secretaría

In [47]:
result = process_speech_turn(turn_test)

In [49]:
result
# save result to json file
with open('embedding_result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)


In [50]:
def embed_full_conference(conference_data, max_tokens=450):
    """
    Procesa todas las intervenciones del discurso en los datos de la conferencia.
    Args:
        conference_data (list): Lista de diccionarios con las intervenciones del discurso.
        max_tokens (int): Número máximo de tokens por chunk.
    Returns:
        list: Lista de diccionarios con embeddings y metadatos para todas las intervenciones.
    """
    all_embeddings = []

    for turn in conference_data:
        embeddings = process_speech_turn(turn, max_tokens)
        all_embeddings.extend(embeddings)

    return all_embeddings

In [51]:
# Embed entire conference
full_conference_embeddings = embed_full_conference(conference_sample, max_tokens=450)
# Save to json file
with open('full_conference_embeddings.json', 'w', encoding='utf-8') as f:
    json.dump(full_conference_embeddings, f, ensure_ascii=False, indent=4) 